In [1]:
from sklearn.model_selection import KFold, train_test_split
import torch.distributions.distribution as D
import data.medain_filtering_class as mf
from torch.utils.data import DataLoader
from torch.autograd import Variable
from new_RBM import new_RBM as RBM
import matplotlib.pyplot as plt
import data.read_samples as rs
import torch.optim as optim
import numpy as np
import datetime
import torch

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 100
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

[INFO] Pre-processing for make clean
2022-05-11 20:46:14.670953 model.py code start


In [2]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
db1_sig, db1_label, db2_sig, db2_label, db3_sig, db3_label = rs.return_list()

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read records file from  ./data/db1/
[RSLT]			 Export records ...
		 ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124']
[INFO]./rdsamp commending start
[IWIP]		rdsamp Converting 100
[IWIP]		rdsamp Converting 101
[IWIP]		rdsamp Converting 102
[IWIP]		rdsamp Converting 103
[IWIP]		rdsamp Converting 104
[IWIP]		rdsamp Converting 105
[IWIP]		rdsamp Converting 106
[IWIP]		rdsamp Converting 107
[IWIP]		rdsamp Converting 108
[IWIP]		rdsamp Converting 109
[IWIP]		rdsamp Converting 111
[IWIP]		rdsamp Converting 112
[IWIP]		rdsamp Converting 113
[IWIP]		rdsamp Converting 114
[IWIP]		rdsamp Converting 115
[IWIP]		rdsamp Converting 116
[IWIP]		rdsamp Converting 117
[IWIP]		rdsamp Converting 118
[IWIP]		rdsamp Converting 119
[IWIP]		rdsamp Converting 121
[IWIP

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    (db1_sig + db2_sig), 
    (db1_label + db2_label),
    test_size=0.33,
    shuffle=True
)

train_dataloader = DataLoader(X_train,
                              batch_size=BATCH_SIZE,
                              num_workers=0, collate_fn=lambda x: x,
                              shuffle=True)

test_dataloader = DataLoader(X_test,
                             batch_size=BATCH_SIZE,
                             shuffle=True)

In [4]:
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

In [5]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    '''First bbrbm'''
    for _, (data) in enumerate(train_dataloader):
        try:
            data = Variable(
                torch.tensor(data, dtype=torch.float32)
            ).uniform_(0, 1)

            # print(data)
            # print(data.size())

            print("Success")
        except ValueError:
            print("Fail")
            pass

        data = torch.tensor(data)
        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        # tensor binary
        vog_first, v1, mt = rbm_first(sample_data)
        
        loss_first = rbm_first.free_energy(vog_first) - rbm_first.free_energy(v1)
        loss_.append(loss_first.data)
        
        first_train_op.zero_grad()
        loss_first.backward()
        first_train_op.step()
    
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_18608/311600057.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  torch.tensor(data, dtype=torch.float32)
C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_18608/311600057.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data)


tensor([[0.9320, 0.1521, 0.7271,  ..., 0.4658, 0.2308, 0.5470],
        [0.9319, 0.9056, 0.5024,  ..., 0.0543, 0.1989, 0.7943],
        [0.8027, 0.5858, 0.2954,  ..., 0.4539, 0.6684, 0.3147],
        ...,
        [0.4659, 0.1627, 0.7806,  ..., 0.4736, 0.9233, 0.4948],
        [0.4482, 0.5186, 0.9083,  ..., 0.1595, 0.7647, 0.3247],
        [0.8725, 0.5597, 0.4831,  ..., 0.7135, 0.0346, 0.9787]])
torch.Size([10, 1300000])
Success


RuntimeError: The expanded size of the tensor (1) must match the existing size (13000000) at non-singleton dimension 1.  Target sizes: [1, 1].  Tensor sizes: [13000000]